In [14]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

import csv
import pandas


In [2]:
df = pandas.read_csv("data/train.csv", index_col="id")
df.head()

D:\Tools\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
id,,,,,,,,,,,,,,,,,,,,,
59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [86]:
nparray = df.values
X_train = nparray[:,1]   #comment_text column
Y_train = nparray[:, 0]   #target column

In [87]:
X_train

array(["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
       "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
       'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
       ..., 'thank you ,,,right or wrong,,, i am following your advice',
       'Anyone who is quoted as having the following exchange, even if apocryphal, would have received my vote! \n\nBessie Braddock: "Winston, you are drunk, and what’s more you are disgustingly drunk."\nWinston Churchill: "Bessie, my dear, you are ugly, and what’s more, you are disgustingly ugly. But tomorrow I shall be sober and you will still be disgustingly ugly."',
       'Students defined as EBD are legally just as disabled and eligible for special services as a developmentally disabled or physically disabled student. \n\nEMOTIONAL AND BEHAVIORAL DISORDER (EBD).\nDefinition. \nAn 

In [88]:
Y_train

array([0.0, 0.0, 0.0, ..., 0.0, 0.6212121212121212, 0.0], dtype=object)

In [89]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [90]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs("data/glove.6B.50d.txt")

In [91]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [9]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+1
            
    
    return X_indices

In [10]:
len(word_to_index)

400000

In [11]:
len(word_to_vec_map)

400000

In [12]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [92]:
from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [93]:
def sentence_to_avg(X, word_to_vec_map):
    
    m = X.shape[0]                                   # number of training examples
    emb_dim = word_to_vec_map["cucumber"].shape[0]    #dimension = 50
    
    X_averages = np.zeros((m,emb_dim))
    
    for i in range(m):
        # Step 1: Split sentence into list of lower case words (≈ 1 line)
        sentence = strip_punctuation(X[i])
        words = sentence.lower().split()

        # Initialize the average word vector, should have the same shape as your word vectors.
        
        vec_sum = np.zeros(emb_dim)
    
        # Step 2: average the word vectors. You can loop over the words in the list "words".
        for w in words:
            vec_sum += word_to_vec_map[w]
        
        avg = vec_sum/len(words)
            
        X_averages[i] = avg
        
        return X_averages

In [67]:
X_averages = np.zeros((10,3))
X_averages
#word_to_vec_map["cucumber"].shape

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [94]:
X_averages = sentence_to_avg(X_train, word_to_vec_map)    # this is of shape (1804874, 50)
#X_averages = np.transpose(X_averages)

In [95]:
X_averages.shape

(1804874, 50)

In [96]:
#Y_train = Y_train.reshape(1, Y_train.shape[0])
Y_train.shape

(1804874,)

In [97]:
Y_train = np.transpose(Y_train)

model = Sequential()
model.add(Dense(50, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_averages, Y_train, epochs=50, batch_size=100, shuffle=True)

Epoch 1/50
1804874/1804874 [==============================] - 32s 18us/step - loss: 0.3480 - acc: 0.7007
Epoch 2/50
1804874/1804874 [==============================] - 31s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 3/50
1804874/1804874 [==============================] - 30s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 4/50
1804874/1804874 [==============================] - 31s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 5/50
1804874/1804874 [==============================] - 31s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 6/50
1804874/1804874 [==============================] - 32s 18us/step - loss: 0.3317 - acc: 0.7007
Epoch 7/50
1804874/1804874 [==============================] - 31s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 8/50
1804874/1804874 [==============================] - 31s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 9/50
1804874/1804874 [==============================] - 31s 17us/step - loss: 0.3317 - acc: 0.7007
Epoch 10/50
1804874/1804874 [==========================

In [98]:
scores = model.evaluate(X_averages, Y_train)

1804874/1804874 [==============================] - 32s 18us/step


In [100]:
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 70.07%
